In [ ]:
# Eqternal packages
import os
import torch
import numpy as np

# Pyplot
import matplotlib.pyplot as plt
import matplotlib as mpl


In [ ]:
# Device
DEVICE = 'cpu'
if torch.cuda.is_available():
    DEVICE = 'cuda'
print(DEVICE)

In [ ]:
# Flow parameters
DIM = 2

Retau = 500. 
H = 1. 
Ub = 0.028 
nu = 5e-06
Re = 5600
rho = Re * nu / (H * Ub)

In [ ]:
# The profile of the periodic hill 
def profile(yy, a):
    y = np.array(yy)
    x = y * 28
    ya = y

    h = np.zeros(len(x))

    for i in range(len(x)):
        if (x[i]>=0) and (x[i]<54) :
            ya[i] *= a
        elif (x[i]>54) and (x[i]<=126) :
            ya[i] -= (54/28.0*(1-a))
        elif (x[i]>126) and (x[i]<=198) :
            ya[i] -= (54/28.0*(1-a))
        elif (x[i]>198) and (x[i]<=252) :
            ya[i] -= (54/28.0*(1-a))
            ya[i] -= (x[i]-198)*(1-a)/28.0
    
    for i in range(len(x)):
        if x[i] > 126.0 :
            x[i] = 252.0 - x[i]

        if (x[i]>=0) and (x[i]<9) :
            h[i] = np.minimum(28., 2.8e+01 + 0.0e+00*x[i] + \
                              6.775070969851e-03*x[i]**2 - 2.124527775800e-03*x[i]**3)
        elif (x[i]>=9) and (x[i]<14) :
            h[i] = 2.507355893131E+01 + 9.754803562315E-01*x[i] - \
                   1.016116352781E-01*x[i]**2 + 1.889794677828E-03*x[i]**3
        elif (x[i]>=14) and (x[i]<20) :
            h[i] = 2.579601052357E+01 + 8.206693007457E-01*x[i] - \
                   9.055370274339E-02*x[i]**2 + 1.626510569859E-03*x[i]**3
        elif (x[i]>=20) and (x[i]<30) :
            h[i] = 4.046435022819E+01 - 1.379581654948E+00*x[i] + \
                   1.945884504128E-02*x[i]**2 - 2.070318932190E-04*x[i]**3
        elif (x[i]>=30) and (x[i]<40) :
            h[i] = 1.792461334664E+01 + 8.743920332081E-01*x[i] - \
                   5.567361123058E-02*x[i]**2 + 6.277731764683E-04*x[i]**3
        elif (x[i]>=40) and (x[i]<=54) :
            h[i] = np.maximum(0., 5.639011190988E+01 - 2.010520359035E+00*x[i] + \
                              1.644919857549E-02*x[i]**2 + 2.674976141766E-05*x[i]**3)
        elif (x[i]>54) and (x[i]<=126) :
            h[i] = 0

    hout = h/28.0
    return ya, hout

In [ ]:
# Parameters for periodic hill
alpha = 0.8
yy = np.arange(0, 9, 0.01)

# wall coordinates
x_p = np.genfromtxt("./dataset/coord.csv", delimiter=',', skip_header=1)[:, 0]
y_p = np.genfromtxt("./dataset/coord.csv", delimiter=',', skip_header=1)[:, 1]

# Wall distance
def wall_distance(x, y, yy = np.arange(0, 9, 0.01), alpha = 1.):
    [ya, hout] = profile(yy, alpha)
    distance = np.zeros_like(x)
    for i in range(len(x)):
        distance[i] = np.min(np.sqrt((x[i]-ya)**2 + (y[i]-hout)**2))
    return distance

d = wall_distance(x_p, y_p, yy, alpha)

In [ ]:
DATA_LENGTH = len(x_p)

S = np.zeros((DATA_LENGTH, 3, 3))
R = np.zeros((DATA_LENGTH, 3, 3))

for i in range(3):
    for j in range(3):
        if i == j == 0:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 0]
            R[:, i, j] = 0
        elif i == j == 1:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 4]
            R[:, i, j] = 0
        elif i == j == 2:
            S[:, i, j] = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 8]
            R[:, i, j] = 0
        elif i == 0 and j == 1 or i == 1 and j == 0:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 1]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 3])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 1]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 3])
        elif i == 0 and j == 2 or i == 2 and j == 0:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 2]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 6])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 2]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 6])
        elif i == 1 and j == 2 or i == 2 and j == 1:
            S[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 5]
                                + np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 7])
            R[:, i, j] = 0.5 * (np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 5]
                                - np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , 7])
                        

# Load data
U = np.genfromtxt("./dataset/U.csv", delimiter=',', skip_header=1)[: , :3]
k = np.genfromtxt("./dataset/k.csv", delimiter=',', skip_header=1)
epsilon = np.genfromtxt("./dataset/epsilon.csv", delimiter=',', skip_header=1)
nut = np.genfromtxt("./dataset/nut.csv", delimiter=',', skip_header=1)

p_gradient = np.genfromtxt("./dataset/grad(p).csv", delimiter=',', skip_header=1)[: , :3]
k_gradient = np.genfromtxt("./dataset/grad(k).csv", delimiter=',', skip_header=1)[: , :3]
U_gradient = np.genfromtxt("./dataset/grad(U).csv", delimiter=',', skip_header=1)[: , :9].reshape(-1, 3, 3)

Q_criteria = np.genfromtxt("./dataset/Q.csv", delimiter=',', skip_header=1)

RS = np.genfromtxt("./dataset/tauDNS.csv", delimiter=',', skip_header=1).reshape(-1, 3, 3)


# Calculate the input features
q1 = Q_criteria
q2 = k
q3 = np.zeros(DATA_LENGTH)
for i in range(DATA_LENGTH):
    if np.sqrt(k[i])*d[i]/(50*nu) < 2:
        q3[i] = np.sqrt(k[i])*d[i]/(50*nu)
    else:
        q3[i] = 2
q4 = U[:, 0]*p_gradient[:, 0] + U[:, 1]*p_gradient[:, 1] + U[:, 2]*p_gradient[:, 2]
q5 = k/epsilon
q6 = np.sqrt(p_gradient[:, 0]*p_gradient[:, 0] + p_gradient[:, 1]*p_gradient[:, 1] + p_gradient[:, 2]*p_gradient[:, 2])
UiUj = np.einsum('bi,bj->bij', U, U)
q7 = np.einsum('bij,bij->b', UiUj, U_gradient)
q7 = np.abs(q7)
q8 = U[:, 0]*k_gradient[:, 0] + U[:, 1]*k_gradient[:, 1] + U[:, 2]*k_gradient[:, 2]


# Stack the input features
q = np.stack((q1, q2, q3, q4, q5, q6, q7, q8), axis=0)
RS_iso = RS - 2/3 * k.reshape(-1, 1, 1) * np.eye(3) + 2.0 * nut.reshape(-1, 1, 1) * S
y = RS_iso

In [ ]:
S = torch.tensor(S)
R = torch.tensor(R)
d = torch.tensor(d)
k = torch.tensor(k)
nut = torch.tensor(nut)

q1 = torch.tensor(q1)
q2 = torch.tensor(q2)
q3 = torch.tensor(q3)
q4 = torch.tensor(q4)
q5 = torch.tensor(q5)
q6 = torch.tensor(q6)
q7 = torch.tensor(q7)
q8 = torch.tensor(q8)

# Calculate the prediction of the Reynolds stress tensor
y_pred = np.zeros((DATA_LENGTH, 3, 3))
y_pred[:, 0, 0] = (q2/((1+q3)+(q5*(q7/q2))))
y_pred[:, 0, 1] = q5*q8/(1+q3+q3**2)
y_pred[:, 1, 0] = y_pred[:, 0, 1]
y_pred[:, 1, 1] = -q2/(2+q3)
y_pred[:, 2, 2] = q2-q2/(1+2*q3-q3**2)


RS_pred = 2/3 * k.reshape(-1, 1, 1) * np.eye(3) - 2.0 * nut.reshape(-1, 1, 1) * S + y_pred
RS_RANS = 2/3 * k.reshape(-1, 1, 1) * np.eye(3) - 2.0 * nut.reshape(-1, 1, 1) * S
RS_DNS = torch.tensor(RS)
RS_error = torch.abs(RS_DNS - RS_pred)


In [ ]:
if DIM == 2:
    index = [0, 1, 4, 8]
elif DIM == 3:
    index = [0, 1, 2, 4, 5, 8]
                    
items = ["nolinearRS_pred","nolinearRS_taeget", "RS_Pred", "RS_RANS", "RS_DNS", "RS_Pred_error", "RS_RANS_error"]
data = [ y_pred, RS_iso, RS_pred, RS_RANS, RS_DNS, RS_error, torch.abs(RS_DNS-RS_RANS)]

from mpl_toolkits.axes_grid1 import make_axes_locatable
fig, ax = plt.subplots(len(index), len(items), figsize=(40,10),)
for i, item in enumerate(items):
    for idq, pos in enumerate(index):
        curr_ax = ax[idq, i]
        # div = make_axes_locatable(curr_ax)
        # cax = div.append_axes("right", size="4%", pad=0.4)
        if "error" in item:
            normcmap = plt.matplotlib.colors.Normalize(vmin=0, vmax=5e-5)
        elif "nolinearRS" in item:
            normcmap = plt.matplotlib.colors.Normalize(vmin=-3e-5, vmax=3e-5)
        else:
            normcmap = plt.matplotlib.colors.Normalize(vmin=-3e-5, vmax=7e-5)
        scatter = curr_ax.scatter(x_p, y_p, c=data[i].reshape(-1,9)[:, pos], cmap = "coolwarm", s=1, norm=normcmap)
        curr_ax.set_title(items[i]+"["+str(pos)+"]")

        # Colorbar
        cbar = plt.colorbar(scatter, ax = curr_ax,)
        # cbar = scatter.colorbar(plt.cm.ScalarMappable(norm=normcmap, cmap="coolwarm"), cax = curr_ax, pad=0.005)
        # cbar.set_label("q[%i]"%(i), labelpad=30)

        # Remove ticks
        curr_ax.set_xticks([])
        curr_ax.set_yticks([])
plt.show()



# Plot the velocity profile

In [ ]:
y_sample_PhySO = []
y_sample_RANS = []
y_sample_DNS = []

U_sample_PhySO = []
U_sample_RANS = []
U_sample_DNS = []
pos = ["05", "15", "25", "35", "45", "55", "65", "75", "85"]
case = "case_1p2"
for i in range(len(pos)):
    y_sample_PhySO.append(np.genfromtxt("./testing/" + case + "/postProcessing/sample/20000/x" + pos[i] + "_U.xy", delimiter=' ', skip_header=1)[:,0])
    y_sample_RANS.append(np.genfromtxt("./cases/" + case + "_ke/postProcessing/sample/20000/x" + pos[i] + "_U.xy", delimiter=' ', skip_header=1)[:,0])
    y_sample_DNS.append(np.genfromtxt("./cases/" + case + "_ke/UDNS/x" + pos[i] + "_UDNS.xy", delimiter=' ', skip_header=1)[:,0])

    U_sample_PhySO.append(np.genfromtxt("./testing/" + case + "/postProcessing/sample/20000/x" + pos[i] + "_U.xy", delimiter=' ', skip_header=1)[:, 1])
    U_sample_RANS.append(np.genfromtxt("./cases/" + case + "_ke/postProcessing/sample/20000/x" + pos[i] + "_U.xy", delimiter=' ', skip_header=1)[:,1])
    U_sample_DNS.append(np.genfromtxt("./cases/" + case + "_ke/UDNS/x" + pos[i] + "_UDNS.xy", delimiter=' ', skip_header=1)[:, 1])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(28, 6))
for i in range(len(pos)):
    curr_ax = ax
    curr_ax.plot(U_sample_DNS[i]/Ub + int(pos[i])/10, y_sample_DNS[i], label="DNS", color="black", linestyle="-", linewidth=2)
    curr_ax.plot(U_sample_PhySO[i]/Ub + int(pos[i])/10, y_sample_PhySO[i], label="PhySO", color="blue", linestyle="-", linewidth=2)
    curr_ax.plot(U_sample_RANS[i]/Ub + int(pos[i])/10, y_sample_RANS[i], label="RANS", color="red", linestyle="-", linewidth=2)


fig.suptitle("Velocity Profile\n(" + case + ")", fontsize=16, fontweight="bold")

ax.set_xlim(0, 10)
ax.set_ylim(0, 3)

ax.set_xlabel("x/h + U/U_b", fontsize=14)
ax.set_ylabel("y/H", fontsize=14)

# Legend
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:3], labels[:3], loc="best")

plt.show()